In [4]:
%load_ext autoreload

In [5]:
import os
print( os.getcwd() )
os.chdir( "/var/genie-in-the-box/src" )
print( os.getcwd() )

/var/genie-in-the-box/src/notebooks
/var/genie-in-the-box/src


In [6]:
%autoreload

import sys
import os
import json


# os.environ[ "LANGCHAIN_WANDB_TRACING" ] = "true"
# # wandb documentation to configure wandb using env variables
# # https://docs.wandb.ai/guides/track/advanced/environment-variables
# # here we are configuring the wandb project name
# os.environ[ "WANDB_PROJECT" ] = "deepily-dataframe-agent"

path = "/var/genie-in-the-box/src/lib"
if path not in sys.path:
    sys.path.append( path )
else:
    print( f"[{path}] already in sys.path" )

print( sys.path )


import lib.util             as du
path = "/var/genie-in-the-box/src"
du.add_to_path( path )

import lib.util_pandas      as up
import lib.util_stopwatch   as sw
import lib.util_code_runner as ucr

from solution_snapshot_mgr import SolutionSnapshotManager
from agent_refactoring     import RefactoringAgent

import openai

debug   = True
verbose = True


['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/var/genie-in-the-box/src/notebooks', '/usr/local/lib/python310.zip', '/usr/local/lib/python3.10', '/usr/local/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/site-packages', '/var/genie-in-the-box/src/lib']
Added [/var/genie-in-the-box/src] to sys.path
pwd [/var/genie-in-the-box/src]

/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug
/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev
/usr/local/lib/python3.10
/usr/local/lib/python3.10/lib-dynload
/usr/local/lib/python3.10/site-packages
/usr/local/lib/python310.zip
/var/genie-in-the-box/src
/var/genie-in-the-box/src/lib
/var/genie-in-the-box/src/notebooks


In [77]:
GPT_4   = "gpt-4-0613"
GPT_3_5 = "gpt-3.5-turbo-0613"

def ask_chat_gpt_text( preamble, query, model=GPT_4, debug=False ):

    openai.api_key = os.getenv( "FALSE_POSITIVE_API_KEY" )
    
    timer = sw.Stopwatch( msg=f"Asking ChatGPT [{model}]...".format( model ) )
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[ 
            { "role": "system", "content": preamble },
            { "role": "user",   "content": query } 
        ],
        temperature=0,
        max_tokens=2400,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    timer.print( use_millis=True )
    
    if debug: print( json.dumps( response, indent=4 ) )
    
    return response[ "choices" ][ 0 ][ "message" ][ "content" ].strip()

preamble = "You are chatGPT, a helpful chatbot"
question = "What's your name?"

ask_chat_gpt_text( preamble, question, model=GPT_4 )

Asking ChatGPT [gpt-4-0613]...
Asking ChatGPT [gpt-4-0613]... in 1,656 ms


"My name is ChatGPT. I'm an artificial intelligence designed to assist with information and conversation."

In [78]:
import pandas as pd
df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )
df[ ( df.event_type == 'Concert' ) ].start_date.value_counts()


start_date
2023-11-01    2
2023-10-28    2
2023-09-30    2
2023-10-13    1
2023-10-30    1
2023-10-22    1
2023-10-21    1
2023-10-20    1
2023-10-19    1
2023-10-15    1
2023-09-01    1
2023-09-06    1
2023-10-03    1
2023-09-28    1
2023-09-27    1
2023-09-20    1
2023-09-12    1
2023-09-09    1
2023-10-07    1
Name: count, dtype: int64

In [76]:
df.event_type.value_counts()

event_type
Conference      24
Concert         22
Performance     21
Appointment     17
Anniversary     17
Workout         17
Birthday        16
Workshop        15
TODO            14
Subscription    14
Meeting         12
Interview       11
Name: count, dtype: int64

In [28]:
%autoreload
    
def refactor_similar_questions( question ):
    
    path_to_snapshots = du.get_project_root() + "/src/conf/long-term-memory/solutions/"
    snapshot_mgr      = SolutionSnapshotManager( path_to_snapshots, debug=False )
    exemplar_snapshot = snapshot_mgr.get_snapshots_by_question( question )[ 0 ][ 1 ]
    similar_snapshots = snapshot_mgr.get_snapshots_by_code_similarity( exemplar_snapshot, threshold=90.0 )
    
    agent = RefactoringAgent( 
        similar_snapshots=similar_snapshots, 
        path_to_solutions="/src/conf/long-term-memory/solutions", 
        debug=True, verbose=True 
    )
    agent.run_prompt()
    
    return agent
    
agent = refactor_similar_questions( "What concerts do I have this week?" )    


------------------------------------------------------------------------------------------------------------------------
- Found [37] synonymous questions
------------------------------------------------------------------------------------------------------------------------

Synonymous question [when is juans birthday] for snapshot.question [when is juans birthday]
Synonymous question [when was juan born] for snapshot.question [when is juans birthday]
Synonymous question [what day is today] for snapshot.question [what day is today]
Synonymous question [whats todays date] for snapshot.question [what day is today]
Synonymous question [whats todays day and date] for snapshot.question [what day is today]
Synonymous question [what is the day and date for today] for snapshot.question [what day is today]
Synonymous question [whats the date] for snapshot.question [what day is today]
Synonymous question [say what day is today] for snapshot.question [what day is today]
Synonymous question [hey

In [79]:
# Print dictionary to console, formatted with indentation, etc.
print( json.dumps( agent.response_dict[ "gpt_function_signatures" ], indent=4 ).replace( "'", '"' ).strip( '"' ) )


[ { "name": "get_this_weeks_events_by_type", "description": "Gets the events of a specific type for the current week", "parameters": { "type": "object", "properties": { "df": { "type": "object", "description": "The DataFrame containing the events data" }, "event_type": { "type": "string", "description": "The type of event to filter by" } }, "required": ["df", "event_type"] } } ]


In [80]:
%autoreload
agent.refactor_code( debug=False )

pwd [/var/genie-in-the-box/io]

/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug
/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev
/usr/local/lib/python3.10
/usr/local/lib/python3.10/lib-dynload
/usr/local/lib/python3.10/site-packages
/usr/local/lib/python310.zip
/var/genie-in-the-box/src
/var/genie-in-the-box/src/lib
/var/genie-in-the-box/src/notebooks
Writing file [/var/genie-in-the-box/src/lib/autogen/util_calendaring_0.py]... Done!
Writing file [/var/genie-in-the-box/src/lib/autogen/util_calendaring_0.json]... Done!
Writing file [/var/genie-in-the-box/io/lib/autogen/util_calendaring_0.py]... Done!

Generating embedding for [import lib.autogen.util_calendaring_0 as uc0 print(uc0.get_this_...]...
Generating embedding for [import lib.autogen.util_calendaring_0 as uc0 print(uc0.get_this_...]... Done! in 181 ms

solution_file value provided: [what-concerts-do-i-have-this-week-0.json]...
File path: /var/genie-in-the-box/src/conf/long-term-memory/s

In [81]:
%autoreload
agent.run_code()

last command, before [print(uc0.get_this_weeks_events_by_type(df, 'Concert'))]:
return_type [string]
ERROR: return_type [string] and last command [print(uc0.get_this_weeks_events_by_type(df, 'Concert'))] are incompatible!
last command,  after [print(uc0.get_this_weeks_events_by_type(df, 'Concert'))]:
import pandas as pd
import lib.util as du
import lib.util_pandas as dup

debug = True

df = pd.read_csv( du.get_project_root() + '/src/conf/long-term-memory/events.csv' )
df = dup.cast_to_datetime( df, debug=debug )
import lib.autogen.util_calendaring_0 as uc0
print(uc0.get_this_weeks_events_by_type(df, 'Concert'))

Executing /var/genie-in-the-box/io/code.py... Done!

------------------------------------------------------------------------------------------------------------------------
- Code output
------------------------------------------------------------------------------------------------------------------------

{"start_date":1697155200000,"end_date":1697155200000,"start_time":"00:

[{'return_code': 0,
  'output': '{"start_date":1697155200000,"end_date":1697155200000,"start_time":"00:00","end_time":"23:59","event_type":"Concert","recurrent":false,"recurrence_interval":null,"priority_level":"highest","name":"Leroy Ruiz","relationship":"father","description_who_what_where":"Concert of Leroy Ruiz at the city center"}\n{"start_date":1697328000000,"end_date":1697328000000,"start_time":"00:00","end_time":"23:59","event_type":"Concert","recurrent":false,"recurrence_interval":null,"priority_level":"high","name":"Tom Ruiz","relationship":"brother","description_who_what_where":"Concert of Tom Ruiz at the city center"}'},
 {'return_code': 0,
  'output': '{"start_date":1696982400000,"end_date":1696982400000,"start_time":"00:00","end_time":"23:59","event_type":"Birthday","recurrent":true,"recurrence_interval":"5 month","priority_level":"highest","name":"Sue","relationship":"sister","description_who_what_where":"Sue\'s birthday party at their favorite bar"}\n{"start_date":16969

### Prototype new function prompt

In [67]:
%autoreload
signature_paths = du.find_files_with_prefix_and_suffix( du.get_project_root() + "/src/lib/autogen/", "util_calendaring", ".json" )
# Given a list of file paths, load each file into a string and append to a list

signatures = du.get_files_as_strings( signature_paths )
signatures = [ json.loads( s ) for s in signatures ]
du.print_list( signatures )

{'name': 'get_this_weeks_events_by_type', 'description': 'Gets the events of a specific type for the current week', 'parameters': {'type': 'object', 'properties': {'df': {'type': 'object', 'description': 'The DataFrame containing the events data'}, 'event_type': {'type': 'string', 'description': 'The type of event to filter by'}}, 'required': ['df', 'event_type']}}


In [75]:
GPT_4   = "gpt-4-0613"
GPT_3_5 = "gpt-3.5-turbo-0613"

def get_function_from_gpt( query, functions, model=GPT_4, debug=False ):

    openai.api_key = os.getenv( "FALSE_POSITIVE_API_KEY" )
    
    timer = sw.Stopwatch( msg=f"Asking ChatGPT [{model}]...".format( model ) )
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[ 
            # { "role": "system", "content": preamble },
            { "role": "user",   "content": query } 
        ],
        functions=functions,
        function_call="auto",
        temperature=0,
        max_tokens=2400,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    timer.print( use_millis=True )
    
    if debug: print( json.dumps( response, indent=4 ) )
    
    message = response[ "choices" ][ 0 ][ "message" ]
    if message.get( "function_call" ):
        function_name = message[ "function_call" ][ "name" ]
        function_args = json.loads( message[ "function_call" ][ "arguments" ] )
        return function_name, function_args
    
    else:
        return None, None

    
    # return response[ "choices" ][ 0 ][ "message" ][ "content" ].strip()

preamble = "You are designed to take a question and map it to a function."
question = "Do I have any appointments today?"

get_function_from_gpt( question, signatures, model=GPT_4, debug=True )

Asking ChatGPT [gpt-4-0613]...
Asking ChatGPT [gpt-4-0613]... in 2,254 ms

{
    "id": "chatcmpl-87neNgwKvp9PGuqxdYQ612lNjzS2Z",
    "object": "chat.completion",
    "created": 1696869843,
    "model": "gpt-4-0613",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "Sure, let me check that for you.",
                "function_call": {
                    "name": "get_this_weeks_events_by_type",
                    "arguments": "{\n  \"event_type\": \"appointment\"\n}"
                }
            },
            "finish_reason": "function_call"
        }
    ],
    "usage": {
        "prompt_tokens": 73,
        "completion_tokens": 31,
        "total_tokens": 104
    }
}


('get_this_weeks_events_by_type', {'event_type': 'appointment'})

In [ ]:
signature_prompt = """

"""